# 1. Notebook Setup

In [9]:
import os
if os.path.split(os.getcwd())[-1].lower() == 'notebooks':
    os.chdir('..')

# 2. Code

In [10]:
from src.resources.command_file import CommandFile, HydroAquifer, HydroSurface, HydroNonSat
from pint import UnitRegistry

ureg = UnitRegistry()
ureg.default_format = '~'

In [11]:
# Build AQUIFER hydro layer
hyd_aq = HydroAquifer()
hyd_aq.write_settings(anisotropy_factor=0.1, log_fluxes=False, print_lithos=False)
hyd_aq.write_omp(False, n_threads=8)
hyd_aq.add_layer (
    name = 'tertiaire',
    path_gw_mesh = 'grid_tert.txt', 
    path_transm_homo = 'Transm_HET_TERT.txt',
    path_storage = 'Emmag_HET_TERT.txt',
    path_h_ini = 'H_ini_HET_TERT.txt',
    path_thick = 'Thickness_HET_TERT.txt',
    path_conductance = 'Cond_TOP_noRIV_TERT.txt',
    path_boundary = 'Bound_TERT.txt',
    mean = 'HARMONIC'
)
hyd_aq.add_layer (
    name = 'craie',
    path_gw_mesh = 'grid_tert.txt', 
    path_transm_homo = 'Transm_HET_CRAI.txt',
    path_storage = 'Emmag_HET_CRAI.txt',
    path_h_ini = 'H_ini_HET_CRAI.txt',
    path_thick = 'Thickness_HET_CRAI.txt',
    path_conductance = 'Cond_TOP_noRIV_CRAI.txt',
    path_boundary = 'Bound_CRAI.txt',
    mean = 'HARMONIC'
)
hyd_aq.add_layer (
    name = 'jurassique',
    path_gw_mesh = 'grid_tert.txt', 
    path_transm_homo = 'Transm_HET_JURA.txt',
    path_storage = 'Emmag_HET_JURA.txt',
    path_h_ini = 'H_ini_HET_JURA.txt',
    path_thick = 'Thickness_HET_JURA.txt',
    path_conductance = 'Cond_TOP_noRIV_JURA.txt',
    path_boundary = 'Bound_JURA.txt',
    mean = 'HARMONIC'
)

# Build SURFACE hydro layer
hyd_surf = HydroSurface()
hyd_surf.write_mto (
    path_mto = '/home/anatole/Documents/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/METEO',
    path_mto_cell = 'MTO_CELLS.txt',
    prefix_rain = 'precip',
    prefix_etp = 'etp',
    format_ = 'UNFORMATTED'
)
hyd_surf.write_wbu (
    catchments = '{ 1 Seine\nTc = [d] 17\ninclude LISTE_CPROD_SEINE.txt }',
    path_donsur = 'DONSUR_BU28.txt',
    path_bu_mto = 'LIEN_BU28_MTO.txt',
    path_cprod = 'LIEN_ELE_BU_CPROD.txt',
    path_cprod_aq = 'LIEN_CPROD_AQ.txt',
    path_cprod_no_aq = 'LIEN_CPROD_BU_NO_AQ.txt',
    val_omp = True,
    n_threads = 8
)
hyd_surf.write_network_description (
    path_network_musk = 'NETWORK_MUSK.txt',
    path_elements_musk = 'ELEMENTS_MUSK.txt',
    ndim = 1,
    calculate_curvature = False,
    dx = 0 * ureg.meter,
    dz = 10 * ureg.meter,
    upstream_hmin = 0. * ureg.meter,
    downstream_hmax = 10. * ureg.meter,
    schem_type = 'MUSKINGUM',
    k_def = 'TTRA',
    val_omp = False,
    n_threads = 8
)

# Build NONSAT hydro layer
hyd_nsat = HydroNonSat()
hyd_nsat.write_settings (
    path_param = 'PARAM_NSAT.txt',
    path_nsat_units = 'LIEN_NSAT_PARAM.txt',
    path_nsat_prod = 'LIEN_NSAT_ELE_BU.txt',
    path_nsat_aq = 'LIEN_NSAT_AQF.txt',
    val_omp = False,
    n_threads = 8
)

In [4]:
input_folders = [
    '/data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/Cmd_files',
    '/data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_HYD',
    '/data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_SURF',
    '/data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_NSAT',
    '/data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_MESH',
    '/data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_AQ',
    '/data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_AQ/Heterogeneous'
]
output_folder = '/data/DATA/DATA_CAWAQS_OUT/TestProj'

cf = CommandFile (
    input_folders = input_folders,
    output_folder = output_folder
)

# TODO: Quantity serialization
cf.add_config (
    name = 'SEINE_SIMPLE',
    year_start = 2000,
    year_stop = 2010,
    dt = 1 * ureg.day,
    transport = False,
    type_ = 'transient',
    eps_Q = '[m3/s] 0.000001',  # 0.000001 * ureg.meter**3 / ureg.second (TODO: print as such)
    eps_Z = '[m] 0.00001',  # 0.00001 * ureg.meter (TODO: print as such)
    eps_pic = '[cm] 15',  # 15. * ureg.cm
    theta = 1.0,
    nit_pic_max = 1,
    print_surf = False,
    debug = False
)

cf.add_hydro_layer_dict (hyd_aq.name, hyd_aq.dict)
cf.add_hydro_layer_dict (hyd_surf.name, hyd_surf.dict)
cf.add_hydro_layer_dict (hyd_nsat.name, hyd_nsat.dict)

cf.add_output (name='MB_AQ',  format='UNFORMATTED', dt=1*ureg.day, active=False, print_final_state=True)
cf.add_output (name='H_AQ',   format='UNFORMATTED', dt=1*ureg.day, active=True, print_final_state=False)
cf.add_output (name='Q_HYD',  format='UNFORMATTED', dt=1*ureg.day, active=True, print_final_state=False)
cf.add_output (name='NSAT',   format='UNFORMATTED', dt=1*ureg.day, active=False)
cf.add_output (name='WATBAL', format='UNFORMATTED', dt=1*ureg.day, active=True, spatial_scale='WATBAL_ELEMENT')

In [5]:
# Build formatted command file
command_file_str = cf.build_command_file(indent_break_max=1)  # cf.command_file

# Write command file to disk
with open ('./data/commFileTest2.COMM', 'w', encoding='utf-8') as f:
    f.write(command_file_str)

print(command_file_str)

Input_folders = /data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/Cmd_files
              = /data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_HYD
              = /data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_SURF
              = /data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_NSAT
              = /data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_MESH
              = /data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_AQ
              = /data/DATA/DATA_CAWAQS/SEINE_3C/DATA_SEINE_SIMPLE/DATA_AQ/Heterogeneous

Output_folder = /data/DATA/DATA_CAWAQS_OUT/TestProj

simulation = {
	SEINE_SIMPLE
	TIME = {
		year_init = 2000
		year_end  = 2010
		dt        = [d] 1
	}
	SETTINGS = {
		transport   = NO
		type        = transient
		eps_Q       = [m3/s] 0.000001
		eps_Z       = [m] 0.00001
		theta       = 1.0
		nit_pic_max = 1
		eps_pic     = [cm] 15
		print_surf  = YES
		debug       = NO
	}
}

HYDRO = {
	AQUIFER = {
		SETTINGS = {
			anisotropy_factor = 0.1
			log_fluxes 